In [71]:
import torch
import torchvision
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [72]:
# dataset definition
class Dataset(Dataset):
    # load the dataset
    def __init__(self):
        # store the inputs and outputs
        self.data = pd.read_csv("/home/semone/db_lisa_tiny/annotations.csv")
        self.filename = data.iloc[:,1].values
        self.sign = data.iloc[:,1:5].values
        self.label = data.iloc[:,5].values

    # number of rows in the dataset
    def __len__(self):
        return len(self.data)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.sign[idx], self.label[idx]]

In [73]:
ds = Dataset()